# Problems of Chapter 2

## 2.1

In [ ]:
from string import ascii_lowercase


def decode_letter(y: str, s: str, alphabet=ascii_lowercase) -> str:
    if y not in alphabet:
        return y
    return alphabet[(alphabet.index(y) - alphabet.index(s)) % len(alphabet)]


ciphertext = "bsaspp kkuosr"
key = "rsidpy dkawoa"
plaintext = "".join(map(decode_letter, ciphertext, key))
print(f"{plaintext =}")

## 2.4

From [wikipedia](https://en.wikipedia.org/wiki/One-time_pad#Perfect_secrecy):

> [...] the ciphertext C gives absolutely no additional information about the plaintext. This is because (intuitively), given a truly uniformly random key that is used only once, a ciphertext can be translated into any plaintext of the same length, and all are equally likely.

## 2.6

If Oscar knows $t$ consecutive plaintext/ciphertext pairs, where $t$ is the period length, he can recover enough key stream bits to decrypt all ciphertext:

$$
\begin{align*}
s_i &= x_i \oplus y_i, & i &= 0, 1, \dots t - 1 & \\
x_i &= s_j \oplus y_i, & i &= t, t + 1, \dots, & j \equiv i \mod t
\end{align*}
$$

## 2.8

In [ ]:
import tabulate

from lfsr import LFSR


def table_of_states_of_lfsr(lfsr: LFSR, num_states: int) -> str:
    """Return a sequence of states of the LFSR formatted as an html table,
    similar to table 2.2 of the book.
    """
    states = []
    it = iter(lfsr)
    for _ in range(num_states):
        state = list(lfsr.registers)
        state.reverse()
        states.append(state)
        next(it)

    headers = [f"FF{i}" for i in range(len(lfsr.registers))]
    headers.append("clk")
    headers.reverse()

    return tabulate.tabulate(states, headers=headers, showindex=True, tablefmt="html")

In [ ]:
# primitive polynomial: x4 + x + 1
table_of_states_of_lfsr(LFSR((1, 1, 0, 0), (0, 0, 0, 1)), 16)

In [ ]:
# reducible polynomial: x4 + x2 + 1
# first sequence:
table_of_states_of_lfsr(LFSR((1, 0, 1, 0), (0, 0, 0, 1)), 7)

In [ ]:
# reducible polynomial: x4 + x2 + 1
# second sequence:
table_of_states_of_lfsr(LFSR((1, 0, 1, 0), (1, 1, 1, 1)), 7)

In [ ]:
# reducible polynomial: x4 + x2 + 1
# third_sequence:
table_of_states_of_lfsr(LFSR((1, 0, 1, 0), (1, 0, 1, 1)), 4)

In [ ]:
# irreducible polynomial: x4 + x3 + x2 + x + 1
# first sequence:
table_of_states_of_lfsr(LFSR((1, 1, 1, 1), (0, 0, 0, 1)), 6)

In [ ]:
# irreducible polynomial: x4 + x3 + x2 + x + 1
# second sequence:
table_of_states_of_lfsr(LFSR((1, 1, 1, 1), (1, 1, 1, 1)), 6)

In [ ]:
# irreducible polynomial: x4 + x3 + x2 + x + 1
# third sequence:
table_of_states_of_lfsr(LFSR((1, 1, 1, 1), (1, 0, 0, 1)), 6)

## 2.10

In [ ]:
import math
from collections.abc import Iterable

import numpy as np
from more_itertools import take, all_equal, chunked

from lfsr import LFSR


def compute_coefficients(key_stream: Iterable[int], degree: int) -> list[int]:
    """Apply a known-plaintext attack to recover the feedback coefficients."""
    key_stream_bits = take(2 * degree, key_stream)

    A = np.array([key_stream_bits[i : i + degree] for i in range(degree)])
    B = np.array(key_stream_bits[degree : 2 * degree])
    x = np.linalg.solve(A, B)

    return (x.astype(int) % 2).tolist()


plaintext = "1001001001101101100100100110"
ciphertext = "1011110000110001001010110001"

key_stream_bits = [int(x) ^ int(y) for x, y in zip(plaintext, ciphertext)]

# we can observe a repeating pattern with a period length of 7:
print(f"{key_stream_bits = }")
assert all_equal(chunked(key_stream_bits, 7))

# assuming maximum-length LFSR:
period_length = 7
degree = int(math.log2(period_length + 1))

iv = key_stream_bits[:degree]
coefficients = compute_coefficients(key_stream_bits, degree)

assert take(len(key_stream_bits), LFSR(coefficients, iv)) == key_stream_bits

print(f"{degree = }")
print(f"{iv = }")
print(f"{coefficients = }")

## 2.11

In [ ]:
from operator import xor

from more_itertools import flatten

from lfsr import LFSR

ALPHABET = ascii_lowercase + "012345"


def encode_letter(letter: str) -> list[int]:
    # z => [1, 1, 0, 1, 0]
    i = ALPHABET.index(letter)
    m = math.ceil(math.log2(len(ALPHABET)))
    bits = []
    for _ in range(m):
        bits.append(i % 2)
        i //= 2
    bits.reverse()
    return bits


def decode_letter(bits: list[int]) -> str:
    # [1, 1, 0, 1, 0] => z
    return ALPHABET[sum(2**i for i, bit in enumerate(reversed(bits)) if bit)]


degree = 6
header = "wpi"
ciphertext = "j5a0edj2b"

header_bits = flatten(map(encode_letter, header))
ciphertext_bits = list(flatten(map(encode_letter, ciphertext)))
key_stream_bits = list(map(xor, header_bits, ciphertext_bits))

iv = key_stream_bits[:degree]
coefficients = compute_coefficients(key_stream_bits, degree)
lfsr = LFSR(coefficients, iv)

plaintext_bits = map(xor, ciphertext_bits, lfsr)
plaintext = "".join(map(decode_letter, chunked(plaintext_bits, 5)))

assert plaintext[:3] == header

print(f"{iv = }")
print(f"{coefficients = }")
print(f"{plaintext = }")

## 2.12

In [ ]:
bits = ""
s = [0] * 285 + [1, 1, 1]

for i in range(70):
    t1 = s[65] ^ s[92]
    t2 = s[161] ^ s[176]
    t3 = s[242] ^ s[287]

    bits += str(t1 ^ t2 ^ t3)

    t1 ^= s[90] & s[91] ^ s[170]
    t2 ^= s[174] & s[175] ^ s[263]
    t3 ^= s[285] & s[286] ^ s[68]

    s[:93] = [t3] + s[:92]
    s[93:177] = [t1] + s[93:176]
    s[177:] = [t2] + s[177:287]

bits